## StackDolly

아래의 StackLLaMA 블로그에서 나온 전체 과정을 SageMaker에서 학습하는 예시입니다. 라이센스 문제로 LLaMA가 아닌 dolly를 활용하여서 진행하도록 합니다.
- StackLLaMA : https://huggingface.co/blog/stackllama
  - PEFT 활용하여 SFT : https://github.com/lvwerra/trl/blob/main/examples/stack_llama/scripts/supervised_finetuning.py
  - reward model 학습 : https://github.com/lvwerra/trl/blob/main/examples/stack_llama/scripts/reward_modeling.py
  - reward model 활용하여 RLHF 까지 모두 진행하는 예시 : https://github.com/lvwerra/trl/blob/main/examples/stack_llama/scripts/rl_training.py
  

In [ ]:
%store -r

In [ ]:
import sagemaker
from sagemaker.utils import name_from_base
from sagemaker.huggingface import HuggingFace

In [ ]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
sm_client = sagemaker_session.sagemaker_client
sm_runtime_client = sagemaker_session.sagemaker_runtime_client

### 학습 진행

DeepSpeed + Accelerate 를 활용한 분산 학습 예시
- SageMaker를 활용해서 mutli-node & multi-gpu로 DeepSpeed + Accelerate하는 예시는 여기를 참고해 주세요: https://github.com/aws-samples/training-llm-on-sagemaker-for-multiple-nodes-with-deepspeed 
- SageMaker 자체도 EC2 instance를 할당 받은 후 학습을 하는 것이기 때문에, 위에서 나온 것 처럼 entrypoint에서는 학습을 위한 스크립트를 별도로 실행하도록 하였고, 이 방법을 활용하면 높은 자유도를 가지고 웬만한 형태의 distributed training이 모두 가능함.

### 코드 작성

- 실제 학습 코드에 관해서는 `stack-dolly-src` 디렉토리를 참고 해 주세요.


In [ ]:
job_name = name_from_base("stack-dolly")
print(job_name)

In [ ]:
# instance_type = 'ml.p3.16xlarge'
instance_type = 'ml.g5.12xlarge'
# instance_type = 'ml.g5.24xlarge'
# instance_type = 'ml.g5.48xlarge'
# instance_type = 'ml.p4d.24xlarge'

instance_count = 1
environment = {'NODE_NUMBER': str(instance_count)}

# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'start.py',          # user endpoint script
    source_dir           = 'stack-dolly-src',               # directory which includes all the files needed for training
    instance_type        = instance_type, # instances type used for the training job
    instance_count       = instance_count,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    transformers_version = '4.26',            # the transformers version used in the training job
    pytorch_version      = '1.13',            # the pytorch_version version used in the training job
    py_version           = 'py39',            # the python version used in the training job
    environment = environment,
)

In [ ]:
pretrained_uri = model_artifact
data = {'pretrained-model': pretrained_uri}

In [ ]:
huggingface_estimator.fit(data, wait=False)


### Next step (Homework)

- SFT 학습이 완료되면, reward modeling 및 RLHF 학습 모두 순차적으로 진행하면 됩니다.
- 기존 `stack-dolly-src` 에 있는 코드를 참고하시면 됩니다.